### Importing the libraries



In [1]:
import numpy as np
import sklearn 
import pandas as pd
import os 
import cv2


from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay

# Loading the images which will serve as ***x***

## *Pictures*

In [2]:
#Loading the pictures and adding them to the X marix
os.chdir(r'/home/guscerra@GU.GU.SE/aics-project/desim_tukstanciu/pictures')

pic_files = os.listdir()

for i, file in enumerate(pic_files[:10000]):

  file_loaded = cv2.imread(file, 0)
  file_vec = np.asarray(file_loaded).reshape(1, -1)

  if i == 0:
    x = file_vec
  else:
    x = np.concatenate((x, file_vec))

print(x.shape)

(10000, 65536)


## *Sketches - high Q*

In [12]:
#Loading the pictures and adding them to the X marix
os.chdir(r'/home/guscerra@GU.GU.SE/aics-project/desim_tukstanciu/sketches')

sketch_files = os.listdir()

for i, file in enumerate(sketch_files[:10000]):

  file_loaded = cv2.imread(file, 0)
  file_vec = np.asarray(file_loaded).reshape(1, -1)

  if i == 0:
    x = file_vec
  else:
    x = np.concatenate((x, file_vec))

print(x.shape)

(10000, 65536)


## *Sketches - low Q*

In [5]:
#Loading the pictures and adding them to the X marix
os.chdir(r'/home/guscerra@GU.GU.SE/aics-project/puikus_sketchai')

sketch_files = os.listdir()

for i, file in enumerate(sketch_files[:10000]):

  file_loaded = cv2.imread(file, 0)
  file_vec = np.asarray(file_loaded).reshape(1, -1)

  if i == 0:
    x = file_vec
  else:
    x = np.concatenate((x, file_vec))

print(x.shape)

(10000, 65536)


# Loading the features which will serve as ***y***

In [3]:
#Accessing the file with features
os.chdir(r'/home/guscerra@GU.GU.SE/aics-project/desim_tukstanciu')

with open('CelebAMask-HQ-attribute-anno.txt', 'r', encoding='utf-8') as infile:
  lines = infile.readlines()

#Extracting columns
features = lines[1].replace('\n', '').split(' ')

#Initialzing a DataFrame
data = pd.DataFrame(columns=features)

#Filling in the DataFrame
for i, line in enumerate(lines[2:10002]):
  split = line.replace('\n', '').split(' ')[2:]
  data_row = [int(variable) for variable in split]
  data.loc[i] = data_row

#Chaning -1 to 0 for convenience
data = data.replace(-1, 0)

#Last touch
y = data.to_numpy()
print(y.shape)

(10000, 40)


Train test split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

print(f'X_train size: {len(X_train)}')
print(f'y_train size: {len(y_train)}\n')
print(f'X_test size: {len(X_test)}')
print(f'y_test size: {len(y_test)}')

X_train size: 9000
y_train size: 9000

X_test size: 1000
y_test size: 1000


In [5]:
#A function to map the prediction to the original features
def prediction_to_features(prediction):
  predicted_features = []

  for i, value in enumerate(prediction[0, :]):
    if value == 1:
      predicted_features.append(features[i])

  return predicted_features

The classifier

In [8]:
forest = RandomForestClassifier(random_state=1)
multi_target_forest = MultiOutputClassifier(forest, n_jobs=-1)

multi_target_forest.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=1),
                      n_jobs=-1)

In [9]:
x_hat = multi_target_forest.predict(X_test[0].reshape(1, -1))

print(f'Predicted features: {prediction_to_features(x_hat)}\n')
print(f'Gold standard: {prediction_to_features(y)}')

Predicted features: ['Attractive', 'Heavy_Makeup', 'No_Beard', 'Smiling', 'Wearing_Lipstick', 'Young']

Gold standard: ['Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Big_Lips', 'Brown_Hair', 'Bushy_Eyebrows', 'Heavy_Makeup', 'High_Cheekbones', 'Mouth_Slightly_Open', 'No_Beard', 'Pointy_Nose', 'Smiling', 'Wavy_Hair', 'Wearing_Lipstick', 'Young']


In [11]:
print(classification_report(y_test, y_hat, target_names=features))


                     precision    recall  f1-score   support

   5_o_Clock_Shadow       0.17      0.01      0.01       141
    Arched_Eyebrows       0.43      0.11      0.18       437
         Attractive       0.59      0.89      0.71       593
    Bags_Under_Eyes       0.38      0.03      0.05       296
               Bald       0.00      0.00      0.00        18
              Bangs       0.33      0.02      0.04       161
           Big_Lips       0.38      0.13      0.19       405
           Big_Nose       0.38      0.04      0.07       328
         Black_Hair       0.31      0.02      0.04       209
         Blond_Hair       0.00      0.00      0.00       205
             Blurry       0.00      0.00      0.00         4
         Brown_Hair       0.12      0.00      0.01       230
     Bushy_Eyebrows       0.50      0.02      0.04       176
             Chubby       0.00      0.00      0.00        68
        Double_Chin       0.00      0.00      0.00        45
         Eyeglasses    

/usr/local/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
y_hat = multi_target_forest.predict(X_test)


## SVM

In [ ]:
svm = LinearSVC(random_state=42)

multi_target_svm = MultiOutputClassifier(svm, n_jobs=-1)


In [ ]:
multi_target_svm.fit(X_train, y_train)

y_hat = multilabel_classifier.predict(X_test)

print(classification_report(y_test, y_hat, target_names=features))